<a href="https://colab.research.google.com/github/bongho/Project_ASAC_5th_Review/blob/hoon/%ED%82%A4%EC%9B%8C%EB%93%9C_%EA%B8%B0%EB%B0%98_%EC%B6%94%EC%B6%9C_%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers==4.17

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 70.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

import torch
import pandas as pd
import numpy as np

# 데이터 준비 및 전처리

In [ ]:
review = pd.read_csv('/content/drive/MyDrive/ASAC 데이터 분석 과정/기업연계_프로젝트/encoded_review_df_filtered.csv')
review.head(1)

df = review

In [ ]:
### 가게 리뷰 groupby 통합

temp_df = df.drop(['review_id','user_id', 'date'], axis=1)
group_temp = temp_df.groupby('business_id').agg({
    'text': ' '.join,
    'stars': ['count','mean'],
    'useful': 'sum',
    'funny': 'sum',
    'cool': 'sum'
    }).reset_index()
group_temp.columns = ['business_id', 'text', 'count','stars','useful', 'funny', 'cool']

## 문장별 텍스트 리스트 컬럼 추가 버전 0805 이후

In [2]:
df = pd.read_csv('/content/drive/MyDrive/ASAC 데이터 분석 과정/기업연계_프로젝트/review_encoding.csv')

In [24]:
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,sentences
0,2348758,1575984,80739,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,"['If you decide to eat here, just be aware it ..."
1,3273772,1231666,98537,4.0,1,0,1,Cute interior and owner gave us tour of upco...,2017-01-14 20:54:15,['Cute interior and owner gave us tour of up...
2,5406063,898460,94651,4.0,1,0,1,I was really between 3 and 4 stars for this on...,2018-07-17 03:30:07,['I was really between 3 and 4 stars for this ...
3,44775,644900,92221,5.0,0,0,0,First time there and it was excellent!!! It fe...,2017-02-19 13:32:05,"['First time there and it was excellent!!!', ""..."
4,3891834,177785,92778,5.0,0,0,0,"Great burgers,fries and salad! Burgers have a...",2017-01-02 03:17:34,"['Great burgers,fries and salad!', 'Burgers ha..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2680161 entries, 0 to 2680160
Data columns (total 9 columns):
 #   Column       Dtype  
---  ------       -----  
 0   review_id    int64  
 1   user_id      int64  
 2   business_id  int64  
 3   stars        float64
 4   useful       int64  
 5   funny        int64  
 6   cool         int64  
 7   text         object 
 8   date         object 
dtypes: float64(1), int64(6), object(2)
memory usage: 184.0+ MB


# Siebert 활용하여 감성(sentiment) 분류

- 성능이 잘나오는 편
- 다양한 데이터셋에서 학습을 해서 분류를 하는 모델임으로 새로운 데이터셋에도 적용하면 잘 분류해줌

In [3]:
from transformers import pipeline
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
b = sentiment_analysis('''Don't be stupid enough to eat here is. It is expensive and crappy.
Just eat at the cafeteria. My friends and myself came to humpty's to have a nice breakfast and it was horrid and a disaster.
the service was fine, the food was DESPICABLE.
My friend ordered the chicken louie the mashed potatoes tasted like acid reflux and the chicken was so rubbery she couldn't eat half of it. The ground beef in the Mexican scrambler was questionable too.
My other friend ordered the crunchy chicken parmesan the pasta was watery and definitely unappealing and the vegetables were cold and clearly frozen. we all feel sick.''')

print(b[0]['label'])
print(b[0]['score'])

In [4]:
# 나중에 불러올 때 사용
import re
import json


def change_str(in_str):
    in_str = re.sub(r"'", '"',in_str )
    in_str = re.sub(r'"', '"',in_str)
    in_str = re.sub(r"True", 'true',in_str )
    in_str = re.sub(r"False", 'false',in_str )
    return in_str

In [5]:
import ast

# 문자열로 된 리스트를 실제 리스트로 변환하는 함수
def parse_list_column(column):
    return column.apply(ast.literal_eval)

# 'sentences' 컬럼을 리스트로 변환
df['sentences'] = parse_list_column(df['sentences'])

In [6]:
df['sentences'][0][1]

'We have tried it multiple times, because I want to like it!'

### 전체 데이터에 적용

In [ ]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
import pickle
import os

# 중간 저장 파일 경로
checkpoint_path = "sentiment_analysis_checkpoint.pkl"

# 진행 상황 체크 및 복구
if os.path.exists(checkpoint_path):
    with open(checkpoint_path, 'rb') as f:
        processed_results = pickle.load(f)
else:
    processed_results = {}

# 진행 상황 시각화 및 감정 분석 수행
for index, row in tqdm(df.iterrows(), total=len(df)):
    review_id = row['review_id']
    if review_id in processed_results:
        continue

    sentences = row['sentences']
    results = []
    for sentence in sentences:
        result = sentiment_analysis(sentence)[0]
        print(result)
        results.append((sentence, result['label'], result['score']))

    processed_results[review_id] = results

    # 중간 저장
    with open(checkpoint_path, 'wb') as f:
        pickle.dump(processed_results, f)

# 결과를 데이터프레임에 반영
df['sentiment_analysis'] = df['review_id'].apply(lambda x: processed_results.get(x, []))

# 결과 출력
import ace_tools as tools; tools.display_dataframe_to_user(name="Sentiment Analysis Results", dataframe=df)
print(df[['review_id', 'sentiment_analysis']])

In [21]:
import torch
# 감정 분석 파이프라인 설정 (GPU 사용)
device = 0 if torch.cuda.is_available() else -1
sentiment_analysis = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english", device=device)

# 중간 저장 파일 경로
checkpoint_path = "sentiment_analysis_checkpoint.pkl"

# 진행 상황 체크 및 복구
if os.path.exists(checkpoint_path):
    with open(checkpoint_path, 'rb') as f:
        processed_results = pickle.load(f)
else:
    processed_results = {}

# 감정 분석 함수
def analyze_sentiment_batch(sentences):
    return sentiment_analysis(sentences)

# 데이터프레임의 각 리뷰에 대해 감정 분석 수행
rows_to_process = [(index, row) for index, row in df.iterrows() if row['review_id'] not in processed_results]

for index, row in tqdm(rows_to_process, total=len(rows_to_process)):
    review_id = row['review_id']
    sentences = row['sentences']
    results = analyze_sentiment_batch(sentences)
    processed_results[review_id] = [(sentence, result['label'], result['score']) for sentence, result in zip(sentences, results)]


KeyboardInterrupt: 

In [20]:
processed_results[297242]

[('Great place to eat.', 'POSITIVE', 0.998853325843811),
 ('My family loves it and they are terrific as far as food quality and service.',
  'POSITIVE',
  0.9989166259765625)]

In [22]:
processed_results

{2348758: [('If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end.',
   'NEGATIVE',
   0.9994986057281494),
  ('We have tried it multiple times, because I want to like it!',
   'POSITIVE',
   0.9985461235046387),
  ("I have been to it's other locations in NJ and never had a bad experience.",
   'POSITIVE',
   0.9983091354370117),
  ('The food is good, but it takes a very long time to come out.',
   'NEGATIVE',
   0.9993442893028259),
  ('The waitstaff is very young, but usually pleasant.',
   'POSITIVE',
   0.9983834028244019),
  ('We have just had too many experiences where we spent way too long waiting.',
   'NEGATIVE',
   0.999495267868042),
  ('We usually opt for another diner or restaurant on the weekends, in order to be done quicker.',
   'NEGATIVE',
   0.9971745014190674)],
 3273772: [('Cute interior and owner   gave us tour of upcoming patio/rooftop area which will be great on beautiful days like today.',
   'POSITIVE',
   0.998804688

# 리뷰 요약 (T5)

- Text to Text Transfer Transformer
- 통일된 접근법: T5는 텍스트를 텍스트로 변환하는 일관된 접근 방식을 사용하여 키워드 생성 등 다양한 작업을 수행
- 다양한 작업 수행 능력: 텍스트 요약, 번역, 질문 생성, 키워드 생성 등 여러 작업을 하나의 모델로 수행

In [ ]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
from multiprocessing import Pool, set_start_method, cpu_count
from tqdm import tqdm
import numpy as np
import torch

# 'spawn' 시작 방법 설정
set_start_method('spawn', force=True)

# 1. 토크나이저와 모델 로드
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
# 2. 텍스트 요약 함수 정의
def summarize_text(text):
    input_text = "summarize: " + text
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    outputs = model.generate(input_ids=input_ids, max_length=150, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

## 결과 확인

In [ ]:
summarize_text(df['text'][0])

'we have tried it multiple times, because I want to like it! the food is good, but it takes a very long time to come out.'

In [ ]:
group_temp.columns = ['business_id', 'text', 'count','stars','useful', 'funny', 'cool']

## 가게 리뷰 합산 요약

In [ ]:
group_temp['text'][2]
## 모델의 metric
# ROGUE
# 원문과 요약의 공통 비율
# ROGUE 1 : 1-gram안에서의 공통 비율

'Server was ignorant, and did the bare minimum. My boyfriend and I had the Swiss chicken wraps, and the lettuce was soggy and old, and the tomatoes were extremely underripe. I brought the issue to the waitresses attention and nothing was done to fix the issue. Overall a not so great experience. The fries & gravy were tolerable, but not enough for us to return. I definitely do not recommend this establishment. My wife and I had some time to spare before going to the Royal Alex hospital so we decided on Humpty\'s just across the street at 9AM. My wife ordered the waffle and I had the Canadian skillet. Coffee was plentiful. Orders arrived after 12 minutes which was ok considering the place was only about 1/4 full. Waffle topped with overly sweet whipped cream and some strawberries. I had a full skillet but 3/4 were potatoes. Couldn\'t really tell how many eggs were in the skillet as there was a very unusual and off tasting yellow sauce. Needed a magnifying glass to find the bacon. I left 

In [ ]:
df['text']

0          If you decide to eat here, just be aware it is...
1          Cute interior and owner (?) gave us tour of up...
2          I was really between 3 and 4 stars for this on...
3          First time there and it was excellent!!! It fe...
4          Great burgers,fries and salad!  Burgers have a...
                                 ...                        
2680156    The supper club is ridiculously expensive. So,...
2680157    Took a colleague here for dinner as we were tr...
2680158    This place never fails the food is absolutely ...
2680159    This is a good pizza option - they deliver thr...
2680160    Located in the 'Walking District' in Nashville...
Name: text, Length: 2680161, dtype: object

# 리뷰 키워드 추출 (T5)

### 과학 아티클 fine-tuning 모델

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("Voicelab/vlt5-base-keywords")
tokenizer = T5Tokenizer.from_pretrained("Voicelab/vlt5-base-keywords")

task_prefix = "Keywords: "
# inputs = [
#     "Christina Katrakis, who spoke to the BBC from Vorokhta in western Ukraine, relays the account of one family, who say Russian soldiers shot at their vehicles while they were leaving their village near Chernobyl in northern Ukraine. She says the cars had white flags and signs saying they were carrying children.",
#     "Decays the learning rate of each parameter group by gamma every step_size epochs. Notice that such decay can happen simultaneously with other changes to the learning rate from outside this scheduler. When last_epoch=-1, sets initial lr as lr.",
#     "Hello, I'd like to order a pizza with salami topping.",
# ]

for sample in df.loc[:10,'text']:
    input_sequences = [task_prefix + sample]
    input_ids = tokenizer(
        input_sequences, return_tensors="pt", truncation=True
    ).input_ids
    output = model.generate(input_ids, no_repeat_ngram_size=3, num_beams=4)
    predicted = tokenizer.decode(output[0], skip_special_tokens=True)
    results.append({"text": sample, "keywords": predicted})


results_df = pd.DataFrame(results)
#    print(sample, "\n --->", predicted)

#First time there and it was excellent!!!
#It feels like your are entering someone's home.
#The waiters there funny and nice. The food come out very quickly and it is phenomenal!!!
#Definitely will be going back to this place.

# ---> food, vegetable, waiters


config.json:   0%|          | 0.00/807 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. 

The food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker. 
 ---> NJ, food, quicker, waitstaff
Cute interior and owner (?) gave us tour of upcoming patio/rooftop area which will be great on beautiful days like today. Cheese curds were very good and very filling. Really like that sandwiches come w salad, esp after eating too many curds! Had the onion, gruyere, tomato sandwich. Wasn't too much cheese which I liked. Needed something else...pepper jelly maybe. Would like to see more menu options added such as salads w fun cheeses. Lots of beer

전체 합치기

In [ ]:
for sample in group_temp.loc[:10,'text']:
    input_sequences = [task_prefix + sample]
    input_ids = tokenizer(
        input_sequences, return_tensors="pt", truncation=True
    ).input_ids
    output = model.generate(input_ids, no_repeat_ngram_size=3, num_beams=4)
    predicted = tokenizer.decode(output[0], skip_special_tokens=True)
    print(sample, "\n --->", predicted, "\n")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Ate here for the 1st time on Saturday 08/07/2020. 
The food was ok, had the Lobster Boil . The Lobster was ok for a soft shelled Lobster. The Crabs were soggy and mushy. The Clams were gummy and snotty . The headed shrimp were large and tasty. All the items were precooked, obviously. The fish spread was decent , also had the Fried Shrimp and Shrimp scampi  which were decent. BYOB ... ... The atmosphere was nice except for the zillion Flies. I'll be back with a six pack Absolutely fantastic!!!!  I'll be back for sure. Fried lobster tail was delicious. Five star for sure. I was sadly disappointed in my experience. We ordered crabs and they were extremely small, watery and over half were bad(sick crabs), we ordered beans and rice which lacked flavor and literally had one bean in it that you can see from the picture. The coleslaw also lacked taste. We ask for grouper nuggets and a few were not cooked all the way. So given the food experience it was nothing like the reviews we read. Why I a

### ✅ 레스토랑 기반 fine-tuning 모델
- NUSTM/restaurant-t5-base

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("NUSTM/restaurant-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("NUSTM/restaurant-t5-base")

input_ids = tokenizer(
   "The pizza here is delicious!!", return_tensors="pt"
).input_ids  # Batch size 1
outputs = model.generate(input_ids=input_ids,num_beams=4)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# T5 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("NUSTM/restaurant-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("NUSTM/restaurant-t5-base")

# 입력 텍스트
input_text = '''Cute interior and owner gave us tour of upcoming patio/rooftop area which will be great on beautiful days like today.
Cheese curds were very good and very filling.
Really like that sandwiches come w salad, esp after eating too many curds!
Had the onion, gruyere, tomato sandwich. Wasn't too much cheese which I liked.
Needed something else...pepper jelly maybe. Would like to see more menu options added such as salads w fun cheeses.
Lots of beer and wine as well as limited cocktails.
Next time I will try one of the draft wines'''

input_ids = tokenizer(input_text, return_tensors="pt").input_ids
# 텍스트 생성 (키워드 추출)
outputs = model.generate(input_ids, num_beams=4)
predicted_keywords = tokenizer.decode(outputs[0], skip_special_tokens=True,)

print(f"Predicted Keywords: {predicted_keywords}")

In [ ]:
input_text = '''If you decide to eat here, \
just be aware it is going to take about 2 hours from beginning to end.
We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience.
The food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant.
We have just had too many experiences where we spent way too long waiting.
We usually opt for another diner or restaurant on the weekends, in order to be done quicker'''

input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# 텍스트 생성 (키워드 추출)
outputs = model.generate(input_ids, num_beams=4)
predicted_keywords = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Predicted Keywords: {predicted_keywords}")

In [ ]:
input_text = "Don't be stupid enough to eat here is. It is expensive and crappy.   Just eat at the cafeteria. My friends and myself came to humpty's to have a nice breakfast and it was horrid and a disaster. the service was fine, the food was DESPICABLE. My friend ordered the chicken louie the mashed potatoes tasted like acid reflux and the chicken was so rubbery she couldn't eat half of it. The ground beef in the Mexican scrambler was questionable too. My other friend ordered the crunchy chicken parmesan the pasta was watery and definitely unappealing and the vegetables were cold and clearly frozen. we all feel sick. "
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# 텍스트 생성 (키워드 추출)
outputs = model.generate(input_ids)
predicted_keywords = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Predicted Keywords: {predicted_keywords}")

Predicted Keywords: humpty's humpty


# 리뷰 키워드 추출(KeyBert)

- 핵심 문장을 먼저 뽑고 여기에서 키워드 추출
- 양방향 문맥 이해: BERT는 양방향 문맥을 고려하여 텍스트를 이해하는 것에 효과적이다.
- 주로 텍스트 분류, 질문 답변, 문장 관계 예측에 사용됨.


[참고] https://iopscience.iop.org/article/10.1088/1742-6596/1992/4/042077/pdf

# Bert + Topic modeling (Bertopic 개념)

In [ ]:
from transformers import BertModel, BertTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Bert 모델과 토크나이저 로드
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def extract_key_sentences(text, num_sentences=3):
    sentences = text.split('.')
    embeddings = []

    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors='pt')
        outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy())

    similarities = cosine_similarity(embeddings)
    key_sentences = sorted(range(len(similarities)), key=lambda i: similarities[i].sum(), reverse=True)[:num_sentences]

    return [sentences[i] for i in key_sentences]

In [ ]:
extract_key_sentences(group_temp['text'][0])

[' \n\nI ordered to go my order was ready in 10 minutes \n\nThe staff was very polite \n\nMy food was hot, fresh and tasty! The potion of coleslaw was huge an my lobster was a nice size',
 " Can't wait to return!! Great little spot and best of all BYOB !!!!\nLove it \n\nIs the food was fresh coat and they brought in the stone crabs that night on ice !  The boiled shrimp are great and the outside seating makes it feel very old Florida which is cool  , oysters Rockefeller was  pretty good",
 " The establishment makes you feel so comfortable and warm you don't want to leave! I had the Cuban Empanadas's and they were delicious  I ordered the baked beans also and they were some of the best I've ever had"]

In [ ]:
extract_key_sentences(df['text'][0])

[' \n\nThe food is good, but it takes a very long time to come out',
 ' We usually opt for another diner or restaurant on the weekends, in order to be done quicker',
 'If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end']

In [ ]:
df['text'][0]

"If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. \n\nThe food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker."

### TF-IDF, TextRank, LDA 사용하여 토픽 추출

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import networkx as nx

# TF-IDF 키워드 추출
def tfidf_keywords(text, num_keywords=5):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform([text])
    indices = X.toarray().argsort()[0, -num_keywords:]
    features = vectorizer.get_feature_names_out()
    return [features[i] for i in indices]

# TextRank 키워드 추출
def textrank_keywords(text, num_keywords=5):
    words = text.split()
    word_graph = nx.Graph()
    for i, word in enumerate(words):
        for j in range(i+1, min(i+5, len(words))):
            word_graph.add_edge(word, words[j])

    scores = nx.pagerank(word_graph)
    ranked_words = sorted(scores, key=scores.get, reverse=True)[:num_keywords]
    return ranked_words

# LDA 키워드 추출
def lda_keywords(text, num_keywords=5):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform([text])
    lda = LatentDirichletAllocation(n_components=1)
    lda.fit(X)
    indices = lda.components_[0].argsort()[-num_keywords:]
    features = vectorizer.get_feature_names_out()
    return [features[i] for i in indices]


import nltk
from nltk.corpus import stopwords

# 불용어 다운로드
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# 최종 키워드 결합
def extract_keywords(text, num_keywords=5):
    key_sentences = extract_key_sentences(text)
    combined_text = ' '.join(key_sentences)
    preprocessed_text = preprocess_text(combined_text)

    tfidf_kw = tfidf_keywords(combined_text, num_keywords)
    textrank_kw = textrank_keywords(combined_text, num_keywords)
    lda_kw = lda_keywords(combined_text, num_keywords)

    return list(set(tfidf_kw) | set(textrank_kw) | set(lda_kw))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# 리뷰 데이터를 LDA를 사용하여 토픽 모델링
def topic_modeling(reviews, num_topics=5, num_keywords=5):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(reviews)

    lda = LatentDirichletAllocation(n_components=num_topics)
    lda.fit(X)

    topics = []
    for idx, topic in enumerate(lda.components_):
        keywords = [vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-num_keywords - 1:-1]]
        topics.append(keywords)

    return topics

#### 불용어 처리 전 예시

In [ ]:
# # 예시 데이터
reviews = [
    "The food was great and the service was excellent.",
    "I love the ambiance but the food was just okay.",
    "The pasta was delicious and the wine selection was superb."
]

# 토픽별 키워드 추출
topics = topic_modeling(reviews, num_topics=3, num_keywords=5)
for idx, topic in enumerate(topics):
    print(f"Topic {idx+1}: {topic}")


extract_keywords(reviews[0])

Topic 1: ['the', 'was', 'wine', 'superb', 'selection']
Topic 2: ['food', 'and', 'the', 'was', 'ambiance']
Topic 3: ['was', 'the', 'food', 'and', 'great']


['food', 'great', 'was', 'and', 'the', 'service']

#### 불용어 처리 후

In [ ]:
# 예시 데이터
reviews = [
    "The food was great and the service was excellent.",
    "I love the ambiance but the food was just okay.",
    "The pasta was delicious and the wine selection was superb."
]

preprocessed_reviews = [preprocess_text(review) for review in reviews]
topics_v2 = topic_modeling(preprocessed_reviews, num_topics=3, num_keywords=5)
for idx, topic in enumerate(topics_v2):
    print(f"Topic {idx+1}: {topic}")

extract_keywords(preprocessed_reviews[0])

Topic 1: ['food', 'love', 'okay', 'ambiance', 'excellent']
Topic 2: ['wine', 'superb', 'selection', 'pasta', 'delicious']
Topic 3: ['service', 'great', 'excellent', 'love', 'ambiance']


['food', 'great', 'excellent', 'service']

#### 실제 리뷰 데이터로 실험

In [ ]:
preprocessed_reviews = [preprocess_text(review) for review in df.loc[:1,'text']]

topics_v2 = topic_modeling(preprocessed_reviews, num_topics=3, num_keywords=5)
for idx, topic in enumerate(topics_v2):
  print(f"Topic {idx+1}: {topic}")

Topic 1: ['like', 'curds', 'cheese', 'come', 'good']
Topic 2: ['waitstaff', 'want', 'decide', 'way', 'done']
Topic 3: ['waitstaff', 'want', 'decide', 'way', 'done']


# 리뷰 키워드 추출(LLAMA) 시도중... 아직 못함

In [ ]:
!git clone https://github.com/meta-llama/llama-models.git

Cloning into 'llama-models'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 162 (delta 47), reused 38 (delta 33), pack-reused 90
Receiving objects: 100% (162/162), 1.31 MiB | 3.60 MiB/s, done.
Resolving deltas: 100% (73/73), done.


In [ ]:
%cd /content/llama-models/models/llama3_1

/content/llama-models/models/llama3_1


In [ ]:
!bash ./download.sh

Enter the URL from email: https://llama3-1.llamameta.net/*?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoibDA1Z2hibTFma21ic3dmbjlmaDVnNGszIiwiUmVzb3VyY2UiOiJodHRwczpcL1wvbGxhbWEzLTEubGxhbWFtZXRhLm5ldFwvKiIsIkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMjMyMTIxNn19fV19&Signature=TiiCoOZQGJawP%7Ep0A-y0FZSok3gVcVe2uEspSP-Ffb2bGE%7E%7EWMQGjI4AiLzFNI3Q1%7EoDkegx2%7E%7E5BO5Xz5KWD4KzknTqevs4XvjvJnPPsDqWEaIDoEilL8n5l4zE%7EFw8r-sG4LVBxY4Q9hXcL7banbzuw75-7eAbQNhWut6QnXQlgcCASx-%7EPYcUrBTQEdJojZJxKGPobvgdwuFNSTk4O8PQP85K9lgEt-5YYfavYAc8uiHz6GHxDV0%7ETZrqjzZbQkNjOdgaXzqF5Ty71Ugg6xZy%7EOn-Ahqg99mc5cyP7bzZeho3e3OXvP8ShQm7JuaX3N1PvV30RmwBpQYVf6SiVQ__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=1032009467888227

 **** Model list ***
 -  meta-llama-3.1-405b
 -  meta-llama-3.1-70b
 -  meta-llama-3.1-8b
 -  meta-llama-guard-3-8b
 -  prompt-guard
Choose the model to download: meta-llama-3.1-70b

 Selected model: meta-llama-3.1-70b 

 **** Available models to download: ***
 -  meta-lla

In [ ]:
### 총 용량이 너무 커서 디스크에 꽉차는 문제가 발생함.
## 용량을 줄여서 업로드 할 방법을 추가적으로 모색해야 함